In [24]:
import pandas as pd
import random

As we aim to study the popularity and success of movies, analyzing the reviews provided by viewers offers valuable insights into a movie's popular success. To facilitate this, we will compile a dataset comprising movie reviews.

Additionally, in this notebook, we will gather a dataset that includes the exact release dates of movies (including the month). This is because our CMU Movie Summary Corpus dataset typically only provides the year of release.

# 1- Rating (reviews) dataset

## MovieLens

[This MovieLens dataset](https://files.grouplens.org/datasets/movielens/ml-25m-README.html) is a widely used dataset in the field of recommender systems, containing millions of movie ratings and tag assignments collected from the MovieLens website. It offers detailed information about movies, users, and their interactions, making it an essential resource for research and development in personalized recommendation algorithms.

### Sanity Check

In [25]:
file_path = 'ml-25m/ratings.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())

   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


In [26]:
# Check for missing values
missing_values = df.isnull().sum()

# Check for duplicate rows
duplicate_rows = df.duplicated().sum()

# Check for illogical values
# We know that ratings should be on a 0-5 scale
invalid_ratings = df[(df['rating'] < 0) | (df['rating'] > 5)]

# Assuming userId and movieId should be positive integers
invalid_userId = df[df['userId'] < 1]
invalid_movieId = df[df['movieId'] < 1]

# Basic Descriptive Statistics
descriptive_stats = df.describe()

# Printing the results
print("Missing Values:\n", missing_values)
print("\nDuplicate Rows:", duplicate_rows)
print("\nInvalid Ratings:\n", invalid_ratings)
print("\nInvalid User IDs:\n", invalid_userId)
print("\nInvalid Movie IDs:\n", invalid_movieId)
print("\nDescriptive Statistics:\n", descriptive_stats)

Missing Values:
 userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Duplicate Rows: 0

Invalid Ratings:
 Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []

Invalid User IDs:
 Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []

Invalid Movie IDs:
 Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []

Descriptive Statistics:
              userId       movieId        rating     timestamp
count  2.500010e+07  2.500010e+07  2.500010e+07  2.500010e+07
mean   8.118928e+04  2.138798e+04  3.533854e+00  1.215601e+09
std    4.679172e+04  3.919886e+04  1.060744e+00  2.268758e+08
min    1.000000e+00  1.000000e+00  5.000000e-01  7.896520e+08
25%    4.051000e+04  1.196000e+03  3.000000e+00  1.011747e+09
50%    8.091400e+04  2.947000e+03  3.500000e+00  1.198868e+09
75%    1.215570e+05  8.623000e+03  4.000000e+00  1.447205e+09
max    1.625410e+05  2.091710e+05  5.000000e+00  1.574328e+09


Missing Values: There are no missing values in any of the columns, which is great as it means we have a complete dataset.

Duplicate Rows: There are no duplicate rows in your dataset.

Invalid Ratings, User IDs, and Movie IDs: There are no entries that fall outside the expected ranges for ratings, user IDs, or movie IDs.

#### Descriptive Statistics:

- The count shows that there are 25,000,100 entries in the dataset (a lot of data).

- The mean rating is approximately 3.53, indicating a moderately high average rating across all movies.

- The std (standard deviation) for ratings is about 1.06, suggesting a reasonable spread of ratings around the mean.

- min and max values for ratings are within the expected range (0.5 to 5.0).


Overall, the dataset appears to be clean and well-structured, with no apparent issues in terms of missing values, duplicates, or illogical entries. This makes it a solid foundation for our analysis.

### Movies' average rating

In this section of the notebook, we focus on refining our movie review dataset to ensure higher data quality and enable more accurate analysis. 

Our first step involves filtering out users who have provided fewer than 10 reviews, as well as movies that have received less than 50 reviews. This is done to concentrate on data that is both reliable and substantial. 

Next, we normalize the user ratings. This step involves subtracting the average rating of each user from their individual ratings, which helps mitigate individual rating biases. 

Finally, we calculate the normalized average rating for each movie and adjust this by adding the overall average rating back. This adjustment provides a more balanced and representative view of each movie's reception. 

This process is crucial for gaining a more accurate and meaningful insight into movie popularity and viewer preferences, forming a solid foundation for our subsequent analysis.

In [27]:
# Filter users with at least 5 reviews
user_review_counts = df['userId'].value_counts()
users_with_5_reviews = user_review_counts[user_review_counts >= 10].index
df_filtered_users = df[df['userId'].isin(users_with_5_reviews)]

# Filter movies with at least 10 reviews
movie_review_counts = df_filtered_users['movieId'].value_counts()
movies_with_10_reviews = movie_review_counts[movie_review_counts >= 50].index
df_filtered = df_filtered_users[df_filtered_users['movieId'].isin(movies_with_10_reviews)]

# Calculate average rating per user
user_avg = df_filtered.groupby('userId')['rating'].mean()

# Normalize ratings
df_filtered['norm_rating'] = df_filtered.apply(lambda x: x['rating'] - user_avg[x['userId']], axis=1)

# Calculate the normalized average rating for each movie
movie_norm_avg = df_filtered.groupby('movieId')['norm_rating'].mean()

# Optional: Add back the overall average rating
overall_avg = df_filtered['rating'].mean()
adjusted_movie_rating = movie_norm_avg + overall_avg

/var/folders/5x/w56lcgf97dd2_7gsjr9xyzz40000gn/T/ipykernel_72509/1562366980.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['norm_rating'] = df_filtered.apply(lambda x: x['rating'] - user_avg[x['userId']], axis=1)


### Replacing movie ids by the movie name

In [28]:
# Convert the Series to a DataFrame
ratings_df = adjusted_movie_rating.reset_index()

# Rename the columns to 'movieId' and 'rating'
ratings_df.columns = ['movieId', 'rating']

# Print the resulting DataFrame
print(ratings_df)

       movieId    rating
0            1  3.799459
1            2  3.260367
2            3  3.179223
3            4  2.937154
4            5  3.098408
...        ...       ...
13171   205383  3.643094
13172   205425  3.758098
13173   206499  3.149143
13174   206805  3.280014
13175   207830  3.454666

[13176 rows x 2 columns]


In [29]:
file_path = 'ml-25m/movies.csv'

# Read the CSV file into a DataFrame
names_df = pd.read_csv(file_path)

In [30]:
names_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [31]:
# Merge the DataFrames on 'movieId'
merged_df = pd.merge(names_df, ratings_df, on='movieId')

# Select only the relevant columns
final_df = merged_df[['movieId', 'title', 'rating']] 

# Print the resulting DataFrame
print(final_df)

       movieId                                   title    rating
0            1                        Toy Story (1995)  3.799459
1            2                          Jumanji (1995)  3.260367
2            3                 Grumpier Old Men (1995)  3.179223
3            4                Waiting to Exhale (1995)  2.937154
4            5      Father of the Bride Part II (1995)  3.098408
...        ...                                     ...       ...
13171   205383  El Camino: A Breaking Bad Movie (2019)  3.643094
13172   205425  Dave Chappelle: Sticks & Stones (2019)  3.758098
13173   206499     Between Two Ferns: The Movie (2019)  3.149143
13174   206805        In the Shadow of the Moon (2019)  3.280014
13175   207830            Terminator: Dark Fate (2019)  3.454666

[13176 rows x 3 columns]


In [32]:
# Remove the year from the 'title' column
final_df['title'] = final_df['title'].str.replace(r" \(\d{4}\)", "", regex=True)

final_df

/var/folders/5x/w56lcgf97dd2_7gsjr9xyzz40000gn/T/ipykernel_72509/2696667130.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['title'] = final_df['title'].str.replace(r" \(\d{4}\)", "", regex=True)


,movieId,title,rating
0,1,Toy Story,3.799459
1,2,Jumanji,3.260367
2,3,Grumpier Old Men,3.179223
3,4,Waiting to Exhale,2.937154
4,5,Father of the Bride Part II,3.098408
...,...,...,...
13171,205383,El Camino: A Breaking Bad Movie,3.643094
13172,205425,Dave Chappelle: Sticks & Stones,3.758098
13173,206499,Between Two Ferns: The Movie,3.149143
13174,206805,In the Shadow of the Moon,3.280014


In [33]:
final_df["title"][0]

'Toy Story'

# 2- Movies exact dates of realease

Our analysis heavily relies on the exact release dates of movies. However, 50% of our dataset only includes the year of release, not the month. Therefore, we will endeavor to extract as many exact release dates as possible from [this Kaggle dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data). 

Additionally, this dataset contains IMDb votes and ratings, which could also prove useful for our analysis.

In [34]:
movies = pd.read_csv("archive/movies_metadata.csv",
                    usecols=["id","title","vote_average","vote_count","release_date"])

In [35]:
movies

,id,release_date,title,vote_average,vote_count
0,862,1995-10-30,Toy Story,7.7,5415.0
1,8844,1995-12-15,Jumanji,6.9,2413.0
2,15602,1995-12-22,Grumpier Old Men,6.5,92.0
3,31357,1995-12-22,Waiting to Exhale,6.1,34.0
4,11862,1995-02-10,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...
45461,439050,NaN,Subdue,4.0,1.0
45462,111109,2011-11-17,Century of Birthing,9.0,3.0
45463,67758,2003-08-01,Betrayal,3.8,6.0
45464,227506,1917-10-21,Satan Triumphant,0.0,0.0


In [36]:
# Display the first few rows of the release_date column
print(movies['release_date'].head())

# Check the data type of the release_date column
print(movies['release_date'].dtype)

# Count the number of missing or NaN values in the release_date column
print(movies['release_date'].isna().sum())

# Check the length of the date strings
print(movies['release_date'].str.len().unique())

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object
object
87
[10. nan  1.  2.]


In [37]:
# Filtering out rows where the release_date is NaN or its length is less than 7
filtered_movies = movies[movies['release_date'].notna()]
filtered_movies = filtered_movies[filtered_movies['release_date'].str.len() >= 7]

In [38]:
# Check for duplicate rows based on 'id' and 'title'
duplicate_ids = filtered_movies['id'].duplicated().sum()
duplicate_titles = filtered_movies['title'].duplicated().sum()

# Check data types
data_types = filtered_movies.dtypes

# Inspect release dates
# Assuming release_date is in string format, you might convert it to datetime if needed
invalid_release_dates = filtered_movies[~filtered_movies['release_date'].str.match(r'\d{4}-\d{2}-\d{2}')].shape[0]

# Validate vote average and count
invalid_vote_average = filtered_movies[(filtered_movies['vote_average'] < 0) | (filtered_movies['vote_average'] > 10)].shape[0]
invalid_vote_count = filtered_movies[filtered_movies['vote_count'] < 0].shape[0]

# Print results
print(f"Duplicate IDs: {duplicate_ids}")
print(f"Duplicate Titles: {duplicate_titles}")
print(f"\nData Types:\n{data_types}")
print(f"\nInvalid Release Dates: {invalid_release_dates}")
print(f"Invalid Vote Averages: {invalid_vote_average}")
print(f"Invalid Vote Counts: {invalid_vote_count}")


Duplicate IDs: 30
Duplicate Titles: 3180

Data Types:
id               object
release_date     object
title            object
vote_average    float64
vote_count      float64
dtype: object

Invalid Release Dates: 0
Invalid Vote Averages: 0
Invalid Vote Counts: 0


In [39]:
# Remove duplicate rows based on 'id'
filtered_movies = filtered_movies.drop_duplicates(subset='id')

# Re-check for duplicates and data types after filtering
duplicate_ids_after = filtered_movies['id'].duplicated().sum()
duplicate_titles_after = filtered_movies['title'].duplicated().sum()
data_types_after = filtered_movies.dtypes

# Print results after filtering
print(f"Duplicate IDs after filtering: {duplicate_ids_after}")
print(f"Duplicate Titles after filtering: {duplicate_titles_after}")
print(f"\nData Types after filtering:\n{data_types_after}")


Duplicate IDs after filtering: 0
Duplicate Titles after filtering: 3150

Data Types after filtering:
id               object
release_date     object
title            object
vote_average    float64
vote_count      float64
dtype: object


The presence of duplicate titles with different IDs in our filtered_movies DataFrame indicates that there might be different versions or releases of the same movie, or it could be a data entry error. To further refine the data, we can check if these duplicates have the same release date. If they do, it's more likely that they are indeed duplicates and should be removed. However, if the release dates differ, they might be different versions or movies with the same title released in different years.

In [40]:
# Group by title and check if there are multiple unique release dates for the same title
duplicates_with_different_dates = filtered_movies.groupby('title').filter(lambda x: len(x['release_date'].unique()) > 1)

# Display these duplicates
print("Duplicates with Different Release Dates:")
print(duplicates_with_different_dates)

# We keep the entry with the highest vote_count for each title:
cleaned_movies = duplicates_with_different_dates.sort_values(by=['title', 'vote_count'], ascending=[True, False])
#cleaned_movies = cleaned_movies.drop_duplicates(subset='title', keep='first')

# Final check for duplicates
final_duplicate_titles = cleaned_movies['title'].duplicated().sum()
print(f"\nFinal Duplicate Titles: {final_duplicate_titles}")

Duplicates with Different Release Dates:
           id release_date                  title  vote_average  vote_count
5         949   1995-12-15                   Heat           7.7      1886.0
6       11860   1995-12-15                Sabrina           6.2       141.0
16       4584   1995-12-13  Sense and Sensibility           7.2       364.0
25      16420   1995-12-15                Othello           7.0        33.0
27      17015   1995-09-27             Persuasion           7.4        36.0
...       ...          ...                    ...           ...         ...
45421   52103   1997-01-01             Pickpocket           7.2        13.0
45453  404604   2017-07-07                    Mom           6.6        14.0
45454  420346   2015-01-11      The Morning After           4.0         2.0
45460   30840   1991-05-13             Robin Hood           5.7        26.0
45463   67758   2003-08-01               Betrayal           3.8         6.0

[5454 rows x 5 columns]

Final Duplicate Title

In [41]:
# Group by 'title' and 'release_date', then sort by 'vote_count' within each group
filtered_movies_sorted = filtered_movies.sort_values(by=['title', 'release_date', 'vote_count'], ascending=[True, True, False])

# Drop duplicates, keeping the first entry in each group (the one with the highest vote count)
cleaned_movies = filtered_movies_sorted.drop_duplicates(subset=['title', 'release_date'], keep='first')

# Final check for duplicates
final_duplicate_titles = cleaned_movies['title'].duplicated().sum()
print(f"Final Duplicate Titles: {final_duplicate_titles}")

Final Duplicate Titles: 3150


In [42]:
cleaned_movies

,id,release_date,title,vote_average,vote_count
18757,55245,2010-01-01,!Women Art Revolution,4.3,2.0
30961,41371,2010-07-27,#1 Cheerleader Camp,3.4,23.0
36153,301325,2015-11-20,#Horror,3.4,53.0
23501,267752,2013-11-21,#chicagoGirl,7.0,1.0
28042,143747,1966-12-18,"$1,000 on the Black",6.0,2.0
...,...,...,...,...,...
22346,388182,2013-08-07,هیچ کجا هیچ کس,10.0,1.0
38463,120249,1974-11-05,‘Rameau’s Nephew’ by Diderot (Thanx to Dennis ...,10.0,1.0
8882,14670,1986-09-23,’Round Midnight,6.7,20.0
39033,109380,1965-02-12,…And the Fifth Horseman Is Fear,8.0,4.0


# 3- Merge all : movie title + movie date of release + ratings
In this section of the notebook, we perform a critical step of merging the movie titles, their respective dates of release, and the ratings. This consolidation allows us to form a comprehensive dataset that we will be using moving forward.

In [43]:
# Find common titles
common_titles = cleaned_movies[cleaned_movies['title'].isin(final_df['title'])]

# Count the number of common titles
num_common_titles = common_titles['title'].nunique()

# Print the number of common titles
print(f"Number of Common Titles: {num_common_titles}")

Number of Common Titles: 8123


In [44]:
# Merge the DataFrames on the 'title' column
# 'how' parameter is set to 'left' to keep all records from metadata_df and only add ratings where available
combined_df = pd.merge(cleaned_movies, final_df[['title', 'rating']], on='title', how='left')

In [45]:
combined_df[combined_df['title'] == 'Toy Story']

,id,release_date,title,vote_average,vote_count,rating
42721,862,1995-10-30,Toy Story,7.7,5415.0,3.799459


In [46]:
# Save the dataframe as .csv
combined_df.to_csv('combined_movies_data.csv', index=False)